# Collection of useful functions for working with ICON data



In [20]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import datetime

In [21]:
def get_datetime(icontime):
    """
    takes a float timevalue from icon data and returns that as a datetime object
    arguments:
        icontime: float containing icon time
    """
    year = int(icontime // 10000)
    month = int(icontime // 100 - year * 100)
    day = int(icontime // 1 - year * 10000 - month * 100)
    date = datetime.datetime(year,month,day)
    time = datetime.timedelta(hours=24*(icontime%1))
    out= date+time
    return out

In [22]:
def get_datetimes(icontimes):
    """
    takes an array of timevalues from netcdf files and creates a datetime object
    arguments: 
        icontimes: 1-D Array of icon floats for time
    """

    out=[]
    for i in icontimes:
        year = int(i // 10000)
        month = int(i // 100 - year * 100)
        day = int(i // 1 - year * 10000 - month * 100)
        date = datetime.datetime(year,month,day)
        time = datetime.timedelta(hours=24*(i%1))
        out.append(date+time)
    return out

In [23]:
def readgrid(path="/work/bb1070/b381019/Input/",name="domain3_DOM02.nc"):
        """
        read Icon grid via xarray, converts radians to degrees
        arguments 
            str(path): path to gridfile
            str(name): name of gridfile
        """     
        rad2deg = 180.0/np.pi
        deg2rad = np.pi/180
        ds_grid = ( xr.open_dataset(path+name))
        # convert grid from radians to degrees
        ds_grid['clon'] = ds_grid['clon']*rad2deg
        ds_grid['clat'] = ds_grid['clat']*rad2deg
        ds_grid['clon_vertices'] = ds_grid['clon_vertices']*rad2deg
        ds_grid['clat_vertices'] = ds_grid['clat_vertices']*rad2deg
        return ds_grid

In [24]:
def NN_gridpoint(lon,lat,grid):
    """
    performs Nearest Neighbor Mapping between grid and specified point
    returns  closest gridpoint index
    arguments:
        lon: longitude of point you want NN mapped on grid
        lat: latitude of point you want NN mapped on grid
        grid: gridfile ,best read in via readgrid() 
    """
    
    return np.argmin((grid["clon"].values[:]-lon)**2+ (grid["clat"].values[:]-lat)**2)

In [25]:
def NN_gridpoints(coordinate,grid):
    """
    performs Nearest Neighbor Mapping between grid and specified points
    returns list of closest gridpoint indices
    arguments:
        grid: gridfile ,best read in via readgrid()
        coordinate:  points np.array([[lon,lat],..,[lon,lat]]) you want the nearest gridpoints of
    """
    gridpoints = np.zeros(len(coordinate))
    
    for i in range(len(coordinate)):
        gridpoints[i] = np.argmin((grid["clon"].values[:]-coordinate[i,0])**2
                                + (grid["clat"].values[:]-coordinate[i,1])**2)
    gridpoints=np.array(gridpoints,dtype=int)
    return gridpoints